# Anime Reccomendation System Initial Cleaning of Data

## Imports

In [2]:
# Imports
from pathlib import Path
import pandas as pd
import numpy as np
#import tensorflow as tf
#import tensorflow_hub as hub
import re
import string

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer,  make_column_selector as selector

#from sklearn.metrics import plot_confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
#from sklearn.metrics import plot_confusion_matrix
#from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import unicodedata
import random
from sklearn.model_selection import GridSearchCV

from transformers import BertTokenizer, BertModel
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from nltk.tokenize import TweetTokenizer
from sklearn.neural_network import MLPClassifier

In [31]:
df_anime = pd.read_csv("../data/animes.csv")
df_profile = pd.read_csv("../data/profiles.csv")
df_review = pd.read_csv("../data/reviews.csv")

## Resetting Value

In [134]:
pd.set_option('display.max_colwidth', None)

In [136]:
pd.reset_option('display.max_colwidth')

## Begin cleaning and inspecting all df's

### First we begin with the anime df

In [7]:
df_anime.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [34]:
df_anime[df_anime["title"] == "Haikyuu!! Second Season"]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
3077,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...


In [ ]:
# huh there are duplicate anime titles? will have to see the whole duped list if we can safely drop

In [35]:
duplicates = df_anime.duplicated('title')
duplicate_rows = df_anime[duplicates]['title']
print(duplicate_rows)

908                               Kengan Ashura 2nd Season
3046                                            Doll Saaya
3065     Bishoujo Animerama: Manami no Michi Tono Sou N...
3066                                       Prima Donna Mai
3067                                 Esper Bishoujo Manami
                               ...                        
19306                                        Flip Flappers
19307                                        Fushigi Yuugi
19308                                     Gakkou no Kaidan
19309          InuYasha Movie 2: Kagami no Naka no Mugenjo
19310             Mobile Suit Gundam: Char's Counterattack
Name: title, Length: 3097, dtype: object


In [36]:
duplicates = df_anime.duplicated('uid')
duplicate_rows = df_anime[duplicates]['uid']
print(duplicate_rows)

908      40269
3046      9317
3065     18483
3066      6903
3067      4833
         ...  
19306    32979
19307      123
19308     1281
19309      450
19310       87
Name: uid, Length: 3095, dtype: int64


In [37]:
df_anime.drop_duplicates(subset = "uid", keep = 'first', inplace = True)

In [38]:
duplicates = df_anime.duplicated('title')
duplicate_rows = df_anime[duplicates]['title']
print(duplicate_rows)

5195              Hinamatsuri
15081    Youkoso! Ecolo Shima
Name: title, dtype: object


In [19]:
df_anime[df_anime['title'] == 'Hinamatsuri']

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
558,36296,Hinamatsuri,While reveling in the successful clinching of ...,"['Comedy', 'Sci-Fi', 'Seinen', 'Slice of Life'...","Apr 6, 2018 to Jun 22, 2018",12.0,254075,390,239.0,8.29,https://cdn.myanimelist.net/images/anime/1580/...,https://myanimelist.net/anime/36296/Hinamatsuri
5195,35102,Hinamatsuri,An educational anime about the history and fou...,"['Historical', 'Kids']",Not available,1.0,774,10059,3596.0,7.05,https://cdn.myanimelist.net/images/anime/2/846...,https://myanimelist.net/anime/35102/Hinamatsuri


In [21]:
temp_df = df_anime[df_anime['title'] == 'Hinamatsuri']

In [26]:
temp_df['link']

558     https://myanimelist.net/anime/36296/Hinamatsuri
5195    https://myanimelist.net/anime/35102/Hinamatsuri
Name: link, dtype: object

In [27]:
temp_df = df_anime[df_anime['title'] == 'Youkoso! Ecolo Shima']

In [28]:
temp_df['link']

14327    https://myanimelist.net/anime/35279/Youkoso_Ecolo_Shima
15081    https://myanimelist.net/anime/39143/Youkoso_Ecolo_Shima
Name: link, dtype: object

- UID 5195 (Hinamatsuri) will be dropped.
    - This is a 1 ep show about the history of Hinamatsuri, no reviews or nothin. Will be dropped
- UID 15081 (Youkoso! Ecolo Shima) will be dropped
    - This uid page doesnt even exist lmao

In [41]:
df_anime.drop_duplicates(subset = "title", keep = 'first', inplace = True)

In [42]:
duplicates = df_anime.duplicated('title')
duplicate_rows = df_anime[duplicates]['title']
print(duplicate_rows)

Series([], Name: title, dtype: object)


In [43]:
duplicates = df_anime.duplicated('link')
duplicate_rows = df_anime[duplicates]['link']
print(duplicate_rows)

Series([], Name: link, dtype: object)


In [56]:
df_anime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16214 entries, 0 to 19006
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         16214 non-null  int64  
 1   title       16214 non-null  object 
 2   synopsis    15451 non-null  object 
 3   genre       16214 non-null  object 
 4   aired       16214 non-null  object 
 5   episodes    15722 non-null  float64
 6   members     16214 non-null  int64  
 7   popularity  16214 non-null  int64  
 8   ranked      14551 non-null  float64
 9   score       15873 non-null  float64
 10  img_url     16049 non-null  object 
 11  link        16214 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 1.6+ MB


In [58]:
#begin inspecting synopsis
df_anime[df_anime['synopsis'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
29,9563,Hidamari Sketch x ☆☆☆ Specials,NaN,"['Slice of Life', 'Comedy', 'School', 'Seinen']","Oct 23, 2010 to Oct 30, 2010",2.0,11600,4073,790.0,7.86,https://cdn.myanimelist.net/images/anime/3/26476.jpg,https://myanimelist.net/anime/9563/Hidamari_Sketch_x_%E2%98%86%E2%98%86%E2%98%86_Specials
31,33280,Gochuumon wa Usagi Desu ka??: Dear My Sister,NaN,"['Slice of Life', 'Comedy']","Nov 11, 2017",1.0,20779,3141,788.0,7.86,https://cdn.myanimelist.net/images/anime/1223/95215.jpg,https://myanimelist.net/anime/33280/Gochuumon_wa_Usagi_Desu_ka__Dear_My_Sister
42,36160,Yaoguai Mingdan 2nd Season,NaN,"['Action', 'Comedy', 'Romance', 'Fantasy']","Aug 2, 2017 to Dec 29, 2017",21.0,5238,5659,777.0,7.87,https://cdn.myanimelist.net/images/anime/4/87461.jpg,https://myanimelist.net/anime/36160/Yaoguai_Mingdan_2nd_Season
212,36370,Ginga Eiyuu Densetsu: Die Neue These - Seiran 2,NaN,"['Action', 'Drama', 'Military', 'Sci-Fi', 'Space']","Oct 25, 2019",4.0,8711,4598,598.0,8.00,https://cdn.myanimelist.net/images/anime/1215/103351.jpg,https://myanimelist.net/anime/36370/Ginga_Eiyuu_Densetsu__Die_Neue_These_-_Seiran_2
216,39565,Boku no Hero Academia the Movie 2: Heroes:Rising,NaN,"['Action', 'Comedy', 'Super Power', 'School', 'Shounen']","Dec 20, 2019",1.0,75929,1408,594.0,7.97,https://cdn.myanimelist.net/images/anime/1019/103292.jpg,https://myanimelist.net/anime/39565/Boku_no_Hero_Academia_the_Movie_2__Heroes_Rising
...,...,...,...,...,...,...,...,...,...,...,...,...
18722,34649,Soukyuu no Fafner: Dead Aggressor - The Beyond,NaN,"['Action', 'Military', 'Sci-Fi', 'Drama', 'Mecha']","May 17, 2019",3.0,4598,5965,2888.0,7.22,https://cdn.myanimelist.net/images/anime/1512/94632.jpg,https://myanimelist.net/anime/34649/Soukyuu_no_Fafner__Dead_Aggressor_-_The_Beyond
18751,39638,Choujigen Game Neptune The Animation: Nep no Natsuyasumi,NaN,"['Action', 'Comedy', 'Fantasy', 'Sci-Fi']","Jul 8, 2019",1.0,7229,4973,2857.0,7.22,https://cdn.myanimelist.net/images/anime/1986/101147.jpg,https://myanimelist.net/anime/39638/Choujigen_Game_Neptune_The_Animation__Nep_no_Natsuyasumi
18783,2634,Doraemon: Doraemon Comes Back,NaN,"['Comedy', 'Fantasy', 'Kids', 'Sci-Fi', 'Shounen']","Jan 3, 1981",1.0,1339,8897,3078.0,7.17,https://cdn.myanimelist.net/images/anime/7/57855.jpg,https://myanimelist.net/anime/2634/Doraemon__Doraemon_Comes_Back
18942,38740,Bai She: Yuan Qi,NaN,"['Action', 'Adventure', 'Historical', 'Demons', 'Drama', 'Romance', 'Martial Arts', 'Fantasy']","Jan 11, 2019",1.0,1246,9117,2904.0,7.21,https://cdn.myanimelist.net/images/anime/1559/103214.jpg,https://myanimelist.net/anime/38740/Bai_She__Yuan_Qi


- Although there are quite a few nulls in the synopsis, I dont believe we have to drop them. 
- Will just fill these in with 'blank' so future models don't break on these nulls

In [65]:
df_anime['synopsis'].fillna(value = 'blank', inplace = True)

In [66]:
df_anime[df_anime['synopsis'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


In [61]:
#begin inspecting episodes
df_anime[df_anime['episodes'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
104,37822,Douluo Dalu 2nd Season,Second part of Douluo Dalu .,"['Action', 'Adventure', 'Historical', 'Superna...","Dec 1, 2018 to ?",NaN,5376,5587,710.0,7.90,https://cdn.myanimelist.net/images/anime/1538/...,https://myanimelist.net/anime/37822/Douluo_Dal...
707,21,One Piece,"Gol D. Roger was known as the ""Pirate King,"" t...","['Action', 'Adventure', 'Comedy', 'Super Power...","Oct 20, 1999 to ?",NaN,948342,35,86.0,8.53,https://cdn.myanimelist.net/images/anime/6/732...,https://myanimelist.net/anime/21/One_Piece
781,235,Detective Conan (TV),"Shinichi Kudou, a high school student of astou...","['Adventure', 'Mystery', 'Comedy', 'Police', '...","Jan 8, 1996 to ?",NaN,196953,545,295.0,8.24,https://cdn.myanimelist.net/images/anime/7/751...,https://myanimelist.net/anime/235/Detective_Co...
1037,39023,Qin Shi Ming Yue: Canghai Hengliu,NaN,"['Action', 'Historical', 'Martial Arts', 'Fant...","Dec 27, 2018 to ?",NaN,553,10761,1051.0,7.74,https://cdn.myanimelist.net/images/anime/1307/...,https://myanimelist.net/anime/39023/Qin_Shi_Mi...
1052,966,Crayon Shin-chan,Just because an anime features a young protago...,"['Slice of Life', 'Comedy', 'Ecchi', 'School',...","Apr 13, 1992 to ?",NaN,44171,2057,1034.0,7.74,https://cdn.myanimelist.net/images/anime/10/59...,https://myanimelist.net/anime/966/Crayon_Shin-...
...,...,...,...,...,...,...,...,...,...,...,...,...
17599,6149,Chibi Maruko-chan (1995),Momoko Sakura is an elementary school student ...,"['Comedy', 'Kids', 'School', 'Slice of Life']","Jan 8, 1995 to ?",NaN,2360,7539,2650.0,7.27,https://cdn.myanimelist.net/images/anime/1108/...,https://myanimelist.net/anime/6149/Chibi_Maruk...
17677,37968,Kirakira Happy★Hirake! Cocotama,NaN,"['Fantasy', 'Kids', 'Magic', 'Slice of Life']","Sep 6, 2018 to Sep 26, 2019",NaN,405,11486,7626.0,6.18,https://cdn.myanimelist.net/images/anime/1742/...,https://myanimelist.net/anime/37968/Kirakira_H...
18819,33443,Luo Xiao Hei Zhan Ji,"The story begins on a rainy night, when a home...","['Comedy', 'Supernatural']","Mar 17, 2011 to ?",NaN,1054,9404,3039.0,7.19,https://cdn.myanimelist.net/images/anime/8/804...,https://myanimelist.net/anime/33443/Luo_Xiao_H...
18888,23539,Gudetama,"Gudetama, an egg that is dead to the world and...","['Comedy', 'Kids']","Mar 31, 2014 to ?",NaN,7241,4942,2959.0,7.20,https://cdn.myanimelist.net/images/anime/10/79...,https://myanimelist.net/anime/23539/Gudetama


In [62]:
df_anime[df_anime['episodes']== 0]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


- Same as synopsis, although these epidoes are unknown, im sure if someone went online and filled these in manually they could get filled in.
- Not my job though, for now will just fill in with 0 (to represent unknown)
- in future models, if anything involving episodes in calculation, I am willing to drop for the model.

In [67]:
df_anime['episodes'].fillna(value = 0, inplace = True)

In [68]:
df_anime[df_anime['episodes'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


In [69]:
#begin inspecting ranked
df_anime[df_anime['ranked'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
1497,30532,Gundam Build Fighters OVA,The official Twitter account for the Gundam B...,"['Action', 'Mecha', 'Sci-Fi']",Not available,1.0,2663,7290,NaN,NaN,https://cdn.myanimelist.net/images/anime/4/732...,https://myanimelist.net/anime/30532/Gundam_Bui...
1498,40242,Idomo,The story starts in a mysterious place in the ...,"['Adventure', 'Fantasy']",2020,1.0,170,14113,NaN,NaN,https://cdn.myanimelist.net/images/anime/1059/...,https://myanimelist.net/anime/40242/Idomo
1499,37962,IDOLiSH7: Second Beat!,Second season of IDOLiSH7 .,['Music'],"Apr, 2020 to ?",0.0,4709,5972,NaN,NaN,https://cdn.myanimelist.net/images/anime/1312/...,https://myanimelist.net/anime/37962/IDOLiSH7__...
1500,40046,ID:Invaded,Sakaido is a genius detective who can track do...,"['Sci-Fi', 'Mystery']","Jan 6, 2020 to ?",0.0,17205,3494,NaN,NaN,https://cdn.myanimelist.net/images/anime/1954/...,https://myanimelist.net/anime/40046/ID_Invaded
1501,39725,I★Chu: Étoile Stage,A famous talent agency called Ailedore has cre...,"['Comedy', 'Music', 'School', 'Slice of Life']",Not available,0.0,626,10645,NaN,NaN,https://cdn.myanimelist.net/images/anime/1102/...,https://myanimelist.net/anime/39725/I%E2%98%85...
...,...,...,...,...,...,...,...,...,...,...,...,...
14982,40358,Gal to Kyouryuu,"One day, Kaede, a gyaru, picks up a dinosaur a...","['Comedy', 'Seinen', 'Slice of Life']","Apr, 2020 to ?",0.0,1479,8997,NaN,NaN,https://cdn.myanimelist.net/images/anime/1428/...,https://myanimelist.net/anime/40358/Gal_to_Kyo...
14983,32455,Gekidol,blank,['Music'],Not available,0.0,837,9906,NaN,NaN,https://cdn.myanimelist.net/images/anime/6/778...,https://myanimelist.net/anime/32455/Gekidol
14984,35380,Gi(a)rlish Number Shura,blank,"['Comedy', 'Parody']",Not available,0.0,4734,5887,NaN,NaN,https://cdn.myanimelist.net/images/anime/13/85...,https://myanimelist.net/anime/35380/Giarlish_N...
15072,39166,Girls & Panzer: Saishuushou Part 3,The third film in the six-part Girls & Panzer...,"['Military', 'School']",Not available,1.0,3355,6797,NaN,NaN,NaN,https://myanimelist.net/anime/39166/Girls___Pa...


In [70]:
df_anime[df_anime['ranked']== 0]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


- Same as above, will fill in with 0

In [71]:
df_anime['ranked'].fillna(value = 0, inplace = True)

In [73]:
df_anime[df_anime['ranked'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


In [74]:
#begin inspecting score
df_anime[df_anime['score'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
1497,30532,Gundam Build Fighters OVA,The official Twitter account for the Gundam B...,"['Action', 'Mecha', 'Sci-Fi']",Not available,1.0,2663,7290,0.0,NaN,https://cdn.myanimelist.net/images/anime/4/732...,https://myanimelist.net/anime/30532/Gundam_Bui...
1498,40242,Idomo,The story starts in a mysterious place in the ...,"['Adventure', 'Fantasy']",2020,1.0,170,14113,0.0,NaN,https://cdn.myanimelist.net/images/anime/1059/...,https://myanimelist.net/anime/40242/Idomo
1499,37962,IDOLiSH7: Second Beat!,Second season of IDOLiSH7 .,['Music'],"Apr, 2020 to ?",0.0,4709,5972,0.0,NaN,https://cdn.myanimelist.net/images/anime/1312/...,https://myanimelist.net/anime/37962/IDOLiSH7__...
1500,40046,ID:Invaded,Sakaido is a genius detective who can track do...,"['Sci-Fi', 'Mystery']","Jan 6, 2020 to ?",0.0,17205,3494,0.0,NaN,https://cdn.myanimelist.net/images/anime/1954/...,https://myanimelist.net/anime/40046/ID_Invaded
1501,39725,I★Chu: Étoile Stage,A famous talent agency called Ailedore has cre...,"['Comedy', 'Music', 'School', 'Slice of Life']",Not available,0.0,626,10645,0.0,NaN,https://cdn.myanimelist.net/images/anime/1102/...,https://myanimelist.net/anime/39725/I%E2%98%85...
...,...,...,...,...,...,...,...,...,...,...,...,...
14982,40358,Gal to Kyouryuu,"One day, Kaede, a gyaru, picks up a dinosaur a...","['Comedy', 'Seinen', 'Slice of Life']","Apr, 2020 to ?",0.0,1479,8997,0.0,NaN,https://cdn.myanimelist.net/images/anime/1428/...,https://myanimelist.net/anime/40358/Gal_to_Kyo...
14983,32455,Gekidol,blank,['Music'],Not available,0.0,837,9906,0.0,NaN,https://cdn.myanimelist.net/images/anime/6/778...,https://myanimelist.net/anime/32455/Gekidol
14984,35380,Gi(a)rlish Number Shura,blank,"['Comedy', 'Parody']",Not available,0.0,4734,5887,0.0,NaN,https://cdn.myanimelist.net/images/anime/13/85...,https://myanimelist.net/anime/35380/Giarlish_N...
15072,39166,Girls & Panzer: Saishuushou Part 3,The third film in the six-part Girls & Panzer...,"['Military', 'School']",Not available,1.0,3355,6797,0.0,NaN,NaN,https://myanimelist.net/anime/39166/Girls___Pa...


In [75]:
df_anime[df_anime['score']== 0]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


- Same as above

In [76]:
df_anime['score'].fillna(value = 0, inplace = True)

In [77]:
df_anime[df_anime['score'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


In [78]:
#begin inspecting img_url
df_anime[df_anime['img_url'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
1572,39518,Vampire in the Garden,"Two girls, Momo and Fine, from different backg...",['Vampire'],Not available,0.0,445,11381,0.0,0.00,NaN,https://myanimelist.net/anime/39518/Vampire_in...
1575,40590,Utawarerumono: Futari no Hakuoro,"After the events of Mask of Deception, the Yam...","['Action', 'Drama', 'Fantasy']",Not available,0.0,4166,6251,0.0,0.00,NaN,https://myanimelist.net/anime/40590/Utawarerum...
1641,34832,Macross (Shin Series),"Macross Delta's second ""Walkūre ga Tomaranai"" ...","['Action', 'Military', 'Sci-Fi', 'Music', 'Spa...",Not available,0.0,3083,6953,0.0,0.00,NaN,https://myanimelist.net/anime/34832/Macross_Sh...
1791,39299,Shoujo Kyouiku RE,blank,['Hentai'],"Jul 5, 2019 to ?",0.0,1606,8456,0.0,6.91,NaN,https://myanimelist.net/anime/39299/Shoujo_Kyo...
1817,32195,Junjou Shoujo Et Cetera Specials,Bonus short episodes,['Hentai'],"Aug 2, 2013 to Oct 4, 2013",2.0,986,9548,0.0,6.39,NaN,https://myanimelist.net/anime/32195/Junjou_Sho...
...,...,...,...,...,...,...,...,...,...,...,...,...
14854,40935,Beastars 2nd Season,Second season of Beastars,"['Slice of Life', 'Psychological', 'Drama', 'S...",Not available,0.0,16035,16321,0.0,0.00,NaN,https://myanimelist.net/anime/40935/Beastars_2...
14973,39519,Dragon's Dogma,Based on a world-famous action RPG set in an o...,"['Action', 'Adventure', 'Fantasy']",Not available,0.0,522,11030,0.0,0.00,NaN,https://myanimelist.net/anime/39519/Dragons_Dogma
15072,39166,Girls & Panzer: Saishuushou Part 3,The third film in the six-part Girls & Panzer...,"['Military', 'School']",Not available,1.0,3355,6797,0.0,0.00,NaN,https://myanimelist.net/anime/39166/Girls___Pa...
16758,38646,King of Prism: Shiny Seven Stars II - Kakeru x...,The second movie in the four-part King of Pri...,"['Music', 'Sports', 'Shoujo']","Mar 23, 2019",1.0,576,10690,7053.0,6.30,NaN,https://myanimelist.net/anime/38646/King_of_Pr...


- Tbh, img_url is just the image. Will just fill the nulls with 'blank' to show no link to picture was provided

In [79]:
df_anime[df_anime['img_url']== 'blank']

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


In [80]:
df_anime['img_url'].fillna(value = 'blank', inplace = True)

In [81]:
df_anime[df_anime['img_url'].isna() == True]

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link


- Some sniff tests below

In [91]:
blank_ranked = df_anime['ranked'] == 0
blank_score = df_anime['score'] == 0
blank_ep = df_anime['episodes'] == 0

In [92]:
len(df_anime[blank_ranked & blank_ep & blank_score])

230

In [93]:
len(df_anime[blank_ranked | blank_ep | blank_score])

1880

In [94]:
len(df_anime[ blank_ranked & blank_score | blank_ep])

603

In [95]:
len(df_anime[blank_ranked | blank_score & blank_ep])

1663

- By itself, ranked is the most missing
- Most common mix of missing is 'score' and 'episodes'


### Cleaning profile df

In [44]:
df_profile.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [47]:
df_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81727 entries, 0 to 81726
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   profile          81727 non-null  object
 1   gender           53856 non-null  object
 2   birthday         46807 non-null  object
 3   favorites_anime  81727 non-null  object
 4   link             81727 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [98]:
duplicates = df_profile.duplicated('profile')
duplicate_rows = df_profile[duplicates]['profile']
print(duplicate_rows)

25792        Chappie7815
25793           VoidHawk
25794           Eoussama
25795           ReLCreed
25796             0Seru0
              ...       
81722         lovelessxd
81723    Shattered_Angel
81724       FluffyWalrus
81725              camco
81726          nyja-chan
Name: profile, Length: 33842, dtype: object


In [51]:
df_profile[df_profile["profile"] == "VoidHawk"]

,profile,gender,birthday,favorites_anime,link
4245,VoidHawk,Male,"Jan 15, 1971","['195', '318', '790', '2217', '1249', '145', '31147']",https://myanimelist.net/profile/VoidHawk
25793,VoidHawk,Male,"Jan 15, 1971","['195', '318', '790', '2217', '1249', '145', '31147']",https://myanimelist.net/profile/VoidHawk
30279,VoidHawk,Male,"Jan 15, 1971","['195', '318', '790', '2217', '1249', '145', '31147']",https://myanimelist.net/profile/VoidHawk
36134,VoidHawk,Male,"Jan 15, 1971","['195', '318', '790', '2217', '1249', '145', '31147']",https://myanimelist.net/profile/VoidHawk


In [55]:
duplicates = df_profile.duplicated('link')
duplicate_rows = df_profile[duplicates]['link']
print(duplicate_rows)

25792        https://myanimelist.net/profile/Chappie7815
25793           https://myanimelist.net/profile/VoidHawk
25794           https://myanimelist.net/profile/Eoussama
25795           https://myanimelist.net/profile/ReLCreed
25796             https://myanimelist.net/profile/0Seru0
                              ...                       
81722         https://myanimelist.net/profile/lovelessxd
81723    https://myanimelist.net/profile/Shattered_Angel
81724       https://myanimelist.net/profile/FluffyWalrus
81725              https://myanimelist.net/profile/camco
81726          https://myanimelist.net/profile/nyja-chan
Name: link, Length: 33842, dtype: object


In [101]:
dupe_user = duplicate_rows.tolist()

In [124]:
random_user = random.choice(dupe_user)
df_profile[df_profile["profile"] == random_user]

,profile,gender,birthday,favorites_anime,link
2632,geekyetti,Female,"Jun 27, 1997",[],https://myanimelist.net/profile/geekyetti
28562,geekyetti,Female,"Jun 27, 1997",[],https://myanimelist.net/profile/geekyetti
40431,geekyetti,Female,"Jun 27, 1997",[],https://myanimelist.net/profile/geekyetti
80558,geekyetti,Female,"Jun 27, 1997",[],https://myanimelist.net/profile/geekyetti


- After looking at many random duped users, came to the conclusion all duped users can be dropped
- no idea why the df has so many duped users but they truly are all dupes, not just name equal but all other values too
- also noticed some users with no 'favorite animes'. These users will prob be dropped in the pipeline later

In [125]:
df_profile.drop_duplicates(subset = "profile", keep = 'first', inplace = True)

In [126]:
df_profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47885 entries, 0 to 77662
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   profile          47885 non-null  object
 1   gender           30878 non-null  object
 2   birthday         26848 non-null  object
 3   favorites_anime  47885 non-null  object
 4   link             47885 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


In [127]:
df_profile[df_profile['birthday'].isna() == True]

,profile,gender,birthday,favorites_anime,link
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
5,eneri,NaN,NaN,"['5114', '4898', '2904', '1575', '1482']",https://myanimelist.net/profile/eneri
9,shadowsplat,NaN,NaN,[],https://myanimelist.net/profile/shadowsplat
12,Kiethol,NaN,NaN,[],https://myanimelist.net/profile/Kiethol
15,12sed,NaN,NaN,"['30', '16201', '22135']",https://myanimelist.net/profile/12sed
...,...,...,...,...,...
77654,Mattierial,NaN,NaN,"['10087', '21327', '585', '28851', '32281', '4...",https://myanimelist.net/profile/Mattierial
77656,daniel1302,NaN,NaN,[],https://myanimelist.net/profile/daniel1302
77659,Officer_Anime,NaN,NaN,[],https://myanimelist.net/profile/Officer_Anime
77660,Yuez,NaN,NaN,[],https://myanimelist.net/profile/Yuez


In [128]:
df_profile[df_profile['gender'].isna() == True]

,profile,gender,birthday,favorites_anime,link
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
5,eneri,NaN,NaN,"['5114', '4898', '2904', '1575', '1482']",https://myanimelist.net/profile/eneri
6,Waffle_Empress,NaN,"May 29, 1996","['338', '322', '440', '199', '28223', '12815',...",https://myanimelist.net/profile/Waffle_Empress
9,shadowsplat,NaN,NaN,[],https://myanimelist.net/profile/shadowsplat
12,Kiethol,NaN,NaN,[],https://myanimelist.net/profile/Kiethol
...,...,...,...,...,...
77654,Mattierial,NaN,NaN,"['10087', '21327', '585', '28851', '32281', '4...",https://myanimelist.net/profile/Mattierial
77656,daniel1302,NaN,NaN,[],https://myanimelist.net/profile/daniel1302
77659,Officer_Anime,NaN,NaN,[],https://myanimelist.net/profile/Officer_Anime
77660,Yuez,NaN,NaN,[],https://myanimelist.net/profile/Yuez


- Seems like I can just fill in gender and birthday with not available

In [129]:
df_profile['gender'].fillna(value = 'unavailable', inplace = True)

In [130]:
df_profile['birthday'].fillna(value = 'unavailable', inplace = True)

In [131]:
df_profile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47885 entries, 0 to 77662
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   profile          47885 non-null  object
 1   gender           47885 non-null  object
 2   birthday         47885 non-null  object
 3   favorites_anime  47885 non-null  object
 4   link             47885 non-null  object
dtypes: object(5)
memory usage: 2.2+ MB


### Cleaning reviews

In [137]:
df_review.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [133]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192112 entries, 0 to 192111
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   uid        192112 non-null  int64 
 1   profile    192112 non-null  object
 2   anime_uid  192112 non-null  int64 
 3   text       192112 non-null  object
 4   score      192112 non-null  int64 
 5   scores     192112 non-null  object
 6   link       192112 non-null  object
dtypes: int64(3), object(4)
memory usage: 10.3+ MB


Just kidding, nothing to clean, already good

## Mapping out how we make the rec system